In [ ]:
import sys
sys.path.insert(0, '/home/jwp/stage/python-fitbit')
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import fitbit

# gather_keys_oauth2.py file needs to be in the same directory. 
# also needs to install cherrypy: https://pypi.org/project/CherryPy/
# pip install CherryPy
import gather_keys_oauth2 as Oauth2
import pandas as pd 
import datetime

# YOU NEED TO PUT IN YOUR OWN CLIENT_ID AND CLIENT_SECRET
CLIENT_ID = 
CLIENT_SECRET = 

In [ ]:
server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

In [ ]:
auth2_client.API_VERSION

In [ ]:
start_time = pd.datetime(year=2020, month=11, day=14)
end_time = pd.datetime.today().date() - datetime.timedelta(days=1)

In [ ]:
def get_heart_rate_data(start_time, end_time):
    """Get data from Fitbit API
    
    Parameters
    ----------
    start_time
    end_time
        
    """
    date_list = []
    df_list = []
    all_dates = pd.date_range(start=start_time, end=end_time)
    # Loop through the dates to compile dataframes
    for one_date in all_dates:
        one_date = one_date.date().strftime("%Y-%m-%d")
        one_date_data = auth2_client.intraday_time_series('activities/heart', 
                                                          base_date=one_date, 
                                                          detail_level='1sec')
        df = pd.DataFrame(one_date_data['activities-heart-intraday']['dataset'])
        date_list.append(one_date)
        df_list.append(df)
    # Loop through dataframes to label with dates
    final_df_list = []
    for date, df in zip(date_list, df_list):
        if len(df) == 0:
            continue    
        df.loc[:, 'date'] = pd.to_datetime(date)
        final_df_list.append(df)
    # Concat compiled dataframes
    final_df = pd.concat(final_df_list, axis = 0)
    
    ## Optional Making of the data have more detailed timestamp (day and hour instead of day)
    hoursDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.hour.apply(lambda x: datetime.timedelta(hours = x))
    minutesDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.minute.apply(lambda x: datetime.timedelta(minutes = x))
    secondsDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.second.apply(lambda x: datetime.timedelta(seconds = x))
    # Getting the date to also have the time of the day
    final_df['date'] = final_df['date'] + hoursDelta + minutesDelta + secondsDelta
    return final_df

In [ ]:
def get_sleep_data(start_time, end_time):
    """Get data from Fitbit API
    
    Parameters
    ----------
    start_time
    end_time
        
    """
    date_list = []
    df_list = []
    stages_df_list = []
    allDates = pd.date_range(start=start_time, end =end_time)
    for oneDate in allDates:
        oneDate = oneDate.date().strftime("%Y-%m-%d")
        oneDayData = auth2_client.sleep(date=oneDate)
        # get number of minutes for each stage of sleep and such. 
        if len(oneDayData['sleep']) == 0:
            continue
        stages_df = pd.DataFrame(oneDayData['summary'])
        df = pd.DataFrame(oneDayData['sleep'][0]['minuteData'])
        date_list.append(oneDate)
        df_list.append(df)
        stages_df_list.append(stages_df)

    final_df_list = []
    final_stages_df_list = []
    for date, df, stages_df in zip(date_list, df_list, stages_df_list):
        if len(df) == 0:
            continue
        df.loc[:, 'date'] = pd.to_datetime(date)
        stages_df.loc[:, 'date'] = pd.to_datetime(date)
        final_df_list.append(df)
        final_stages_df_list.append(stages_df)
    final_df = pd.concat(final_df_list, axis = 0)
    ## Optional Making of the data have more detailed timestamp (day and hour instead of day)
    hoursDelta = pd.to_datetime(final_df.loc[:, 'dateTime']).dt.hour.apply(lambda x: datetime.timedelta(hours = x))
    minutesDelta = pd.to_datetime(final_df.loc[:, 'dateTime']).dt.minute.apply(lambda x: datetime.timedelta(minutes = x))
    secondsDelta = pd.to_datetime(final_df.loc[:, 'dateTime']).dt.second.apply(lambda x: datetime.timedelta(seconds = x))
    # Getting the date to also have the time of the day
    final_df['date'] = final_df['date'] + hoursDelta + minutesDelta + secondsDelta
    final_stages_df = pd.concat(final_stages_df_list, axis = 0)
    return final_df, final_stages_df

In [ ]:
def get_sleep_data_recent(start_time, end_time):
    """Get data from Fitbit API v1.2
    
    Parameters
    ----------
    start_time
    end_time
        
    """
    date_list = []
    df_list = []
    stages_df_list = []
    allDates = pd.date_range(start=start_time, end =end_time)
    for oneDate in allDates:
        oneDate = oneDate.date().strftime("%Y-%m-%d")
        oneDayData = auth2_client.sleep(date=oneDate)
        # get number of minutes for each stage of sleep and such.
        if len(oneDayData['sleep']) == 0:
            continue
        #print(oneDayData['summary'])
        #print("=====")
        #print(oneDayData['sleep'][0]['levels']['data'])
        #break
        stages_df = pd.DataFrame(oneDayData['summary'])
        df = pd.DataFrame(oneDayData['sleep'][0]['levels']['data'])
        date_list.append(oneDate)
        df_list.append(df)
        stages_df_list.append(stages_df)

    final_df_list = []
    final_stages_df_list = []
    for date, df, stages_df in zip(date_list, df_list, stages_df_list):
        if len(df) == 0:
            continue
        df.loc[:, 'date'] = pd.to_datetime(date)
        stages_df.loc[:, 'date'] = pd.to_datetime(date)
        final_df_list.append(df)
        final_stages_df_list.append(stages_df)
    final_df = pd.concat(final_df_list, axis = 0)
    ## Optional Making of the data have more detailed timestamp (day and hour instead of day)
    hoursDelta = pd.to_datetime(final_df.loc[:, 'dateTime']).dt.hour.apply(lambda x: datetime.timedelta(hours = x))
    minutesDelta = pd.to_datetime(final_df.loc[:, 'dateTime']).dt.minute.apply(lambda x: datetime.timedelta(minutes = x))
    secondsDelta = pd.to_datetime(final_df.loc[:, 'dateTime']).dt.second.apply(lambda x: datetime.timedelta(seconds = x))
    # Getting the date to also have the time of the day
    final_df['date'] = final_df['date'] + hoursDelta + minutesDelta + secondsDelta
    final_stages_df = pd.concat(final_stages_df_list, axis = 0)
    return final_df, final_stages_df

In [ ]:
#sleep_recent, sleep_stages_recent = get_sleep_data_recent(start_time, end_time)
sleep_recent.to_csv('sleep_recent.csv', index=None)
sleep_stages_recent.to_csv('sleep_stages_recent.csv', index=None)

In [ ]:
sleep_recent['level'].unique()

In [ ]:
level_map = {'awake': 4, 'wake': 4, 'rem': 3, 'light': 2, 'deep': 1, 'asleep': -1}
sleep_recent['level'] = sleep_recent['level'].map(level_map)

In [ ]:
#heart_rate = get_heart_rate_data(start_time, end_time)
#heart_rate.to_csv('heart_rate.csv', index=None)
heart_rate = pd.read_csv('heart_rate.csv', index_col=None)
heart_rate['date'] = pd.to_datetime(heart_rate['date'])

In [ ]:
heart_rate.head()

In [ ]:
sleep, sleep_stages = get_sleep_data(start_time, end_time)
sleep.to_csv('sleep.csv', index=None)
sleep_stages.to_csv('sleep_stages.csv', index=None)
sleep = pd.read_csv('sleep.csv', index_col=None)
sleep['date'] = pd.to_datetime(sleep['date'])

In [ ]:
plot_start = pd.datetime(year=2020, month=11, day=23)
plot_end = pd.datetime(year=2020, month=12, day=26)

two_days_hr = heart_rate.loc[heart_rate.loc[:, 'date'].between(plot_start, plot_end), :]
two_days_sl = sleep_recent.loc[sleep_recent.loc[:, 'date'].between(plot_start, plot_end), :]

In [ ]:
sleep_recent.dtypes

In [ ]:
two_days_hr.plot('date', 'value')
two_days_sl.plot('date', 'level')
plt.legend('')

In [ ]:
joined = two_days_hr.merge(two_days_sl, left_on='date', right_on='date', suffixes=('_hr', '_sl'), how='inner')

In [ ]:
joined.shape

In [ ]:
import matplotlib
plt.close('all')
fig, ax = plt.subplots(figsize=(14, 7))

dates = matplotlib.dates.date2num(joined.date.values)
ax.plot_date(dates, joined.level.values, marker='.')
ax.plot_date(dates, joined.value.values, marker='.')


In [ ]:
plt.scatter(joined.level.values, joined.value.values)

In [ ]:
joined.loc[joined['level']==1, 'value'].values

In [ ]:
data = [joined.loc[joined['level']==l, 'value'].values for l in [1, 2, 3, 4]]
fig7, ax7 = plt.subplots()
ax7.set_title('Heart rate binned by sleep stages', fontsize=15)
ax7.boxplot(data)
ax7.set_xticks([1, 2, 3, 4])
ax7.set_xticklabels(['Deep', 'Light', 'REM', 'Awake'], rotation=45, fontsize=15)
ax7.set_ylabel('Heart rate (bpm)', fontsize=15)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

# Taken from: https://stackoverflow.com/questions/16266019/python-pandas-group-datetime-column-into-hour-and-minute-aggregations
times = pd.to_datetime(two_days_hr['date'])
two_days_hr.groupby([times.dt.date,times.dt.hour]).value.mean().plot(ax=ax)
two_days_sl.groupby([times.dt.date,times.dt.hour]).value.max().plot(ax=ax)

ax.grid(True,
    axis = 'both',
    zorder = 0,
    linestyle = ':',
    color = 'k')
ax.tick_params(axis = 'both', rotation=45, labelsize=20)
ax.set_xlabel('Date, Hour', fontsize=24)
ax.set_ylabel('Heart Rate', fontsize=24)
fig.tight_layout()
#fig.savefig('coupledaysavergedByMin.png', format = 'png', dpi = 300)